In [2]:
import cirq
from cirq.circuits import Circuit
import numpy as np
from scipy.stats import unitary_group
from cirq.sim import Simulator
sim = Simulator()

# Step 1: Decompose Unitary into 2-level Matrices

In [30]:
# useful matrices
Identity_22 = np.eye(2, dtype=np.complex128)
Pauli_x = np.array([[0, 1], [1, 0]], dtype=np.complex128)

# threshold
thr = 10**-9

def is_unitary(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    A = np.array(A)
    return np.allclose(np.eye(n), A @ A.conj().T)


def is_identity(A):
    n = A.shape[0]
    if (A.shape != (n, n)):
        raise ValueError("Matrix is not square.")
    return np.allclose(A, np.eye(n))


def elimination_matrix(a,b):
    # a, b allowed to be complex
    
    # impose theta real + positive {eq.10}
    theta = np.arctan(abs(b/a))
    
    # lambda is the negative arg() of a
    lamda = - np.angle(a)
    
    # {eq.12}
    mu = np.pi + np.angle(b)
    
    # {eq.7}
    U_special = np.array([ [np.exp(1j*lamda) * np.cos(theta), np.exp(1j*mu) * np.sin(theta)],
                           [-np.exp(-1j*mu) * np.sin(theta), np.exp(-1j*lamda) * np.cos(theta)] ])
    
    return U_special


def two_level_decomp(A):
    
    """
    decomp - list of 2x2 unitaries that decompose A. To reconstruct A, the list of unitaries
             in decomp must be reversed
             
    indices - list of non-trivial rows on which matrices in decomp act. For example, [1,3] indicates
              that row 1 and 3 are non-trivial; all other elements are 1.
    """
    
    n = A.shape[0]
    decomp = []
    indices = []
    A_c = np.copy(A)

    for i in range(n-2):
        for j in range(n-1, i, -1):

            a = A_c[i,j-1]
            b = A_c[i,j]

            # --- need checks --- 
            # if A[i,j] = 0, nothing to do! Except in last row - need to check diagonal element is 1 
            if abs(A_c[i,j]) < thr:
                U_22 = Identity_22

                if j == i+1:
                    U_22 = np.array([[1 / a, 0], [0, a]])

            # if A[i,j-1] = 0, need to swap columns - again checking last row to ensure diagonal element is 1 
            elif abs(A_c[i,j-1]) < thr:
                U_22 = Pauli_x

                if j == i+1:
                    U_22 = np.array([[1 / b, 0], [0, b]])

            # Special unitary matrix
            else: 
                U_22 = elimination_matrix(a,b)

            # ----- U_22 found -----

            # multiply submatrix of A with U_22
            A_c[:,(j-1,j)] = A_c[:,(j-1,j)] @ U_22

            # If not the identity matrix - represents a gate! So should store
            if not is_identity(U_22):
                decomp.append(U_22.conj().T)
                indices.append(np.array([j-1,j]))


        # check for diagonal element equal to 1
        assert np.allclose(A_c[i,i],1.0)
    
    # lower right hand 2x2 matrix remaining after decomp
    lower_rh_matrix = A_c[n-2:n, n-2:n]
    
    # if not equal to I - is a non trivial gate
    if not is_identity(lower_rh_matrix):
        decomp.append(lower_rh_matrix)
        indices.append(np.array([n-2,n-1]))

    return decomp, indices


def gray_method(A):
    
    n = A.shape[0]
    M = np.copy(A)

    # using bitwise_xor find Gray permutations
    permutations = []
    for i in range(n):
        permutations.append(i ^ (i // 2))
        
    # 
    M[:,:] = M[:,permutations]
    M[:,:] = M[permutations,:]
    
    decomp, indices = two_level_decomp(M)
    new_decomp = []
    new_ind = []

    for i in range(len(indices)):
        
        t = np.take(permutations, indices[i])
        if t[0]>t[1]:
            new_decomp.append(decomp[i].T)
            new_ind.append(np.sort(t))

        else:
            new_decomp.append(decomp[i])
            new_ind.append(t)
            
    return new_decomp, new_ind

def hide_toggle(for_next=False):
    import random
    from IPython.display import HTML
    this_cell = """$('div.cell.code_cell.rendered.selected')""" ; next_cell = this_cell + '.next()';
    toggle_text = 'Code show/hide'  # text shown on toggle link
    target_cell = this_cell ;  js_hide_current = '' 

    if for_next:
        target_cell = next_cell; toggle_text += ' next cell';
        js_hide_current = this_cell + '.find("div.input").hide();'
    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """<script>
            function {f_name}() {{{cell_selector}.find('div.input').toggle(); }}
            {js_hide_current}
        </script>
        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(f_name=js_f_name,cell_selector=target_cell,js_hide_current=js_hide_current, toggle_text=toggle_text )
    return HTML(html)

        
hide_toggle()

In [106]:
# Example
num_qubits = 4
qubits =[cirq.GridQubit(0,i) for i in range(num_qubits)]

A = unitary_group.rvs(2**num_qubits)

matrices, indices_list = two_level_decomp(A)

# TODO: apply recomposition test

# Step 2a: Convert indices into sequence of states (Gray's method)

In [99]:
def dec_to_bin(x, num_qubits):
    """
    x - decimal number to convert to binary
    num_qubits - number of qubits
    """

    dec_string = bin(x).replace("0b","")[::-1]
    x_bin = np.zeros(num_qubits, dtype='int')

    j = num_qubits - 1
    for i in dec_string:
        x_bin[j] = int(i)
        j = j -1

    return x_bin

def create_gray_arrs(indices, num_qubits):
    
    """
    Return grey code connecting indx1 to indx2 for an n qubit unitary
    indices
    num_qubits - number of qubits
    """
    
    b1 = dec_to_bin(indices[0], num_qubits)
    b2 = dec_to_bin(indices[1], num_qubits)
    s = np.copy(b1)
    
    code = np.array([b1])

    for i in reversed(range(num_qubits)):

        if s[i] != b2[i]:
            
            s[i] =  (s[i] + 1)%2
            code = np.append(code,[s],axis=0)
        
    return code

hide_toggle()

In [108]:
indices_list[1]

array([13, 14])

In [ ]:
13 =  1101 -> Seq Gate - > 14 = 1110


In [107]:
create_gray_arrs(indices_list[1], num_qubits)

array([[1, 1, 0, 1],
       [1, 1, 0, 0],
       [1, 1, 1, 0]])

In [100]:
# example
set([len(create_gray_arrs(i, num_qubits)) for i in indices_list])

{2, 3, 4, 5}

# Step 2b: Convert two-level matrices to FCGates

In [123]:
class FCGate:

    def __init__(self, control_0, control_1, target, sub_gate):
        
        self.control_0 = control_0
        self.control_1 = control_1
        self.target = target
        self.sub_gate = sub_gate
                
        
    def build_op(self, qubits):
        
        # Create Fully Controlled Gate
        num_controls = len(self.control_0) + len(self.control_1)
        control_values = [0] * len(self.control_0) + [1] * len(self.control_1) 
        fc_gate = cirq.ControlledGate(self.sub_gate, num_controls, control_values)

        # Create ordered qubit list for cirq.ControlledGate: [c0,...,c1,...,target]
        control_0_qubits = [qubits[i] for i in self.control_0]
        control_1_qubits = [qubits[i] for i in self.control_1]
        target_qubit = [qubits[self.target]]
        ordered_qubits = control_0_qubits + control_1_qubits + target_qubit

        
        return fc_gate.on(*ordered_qubits) 
    
def state1_to_state2_gate(arr1, arr2, sub_gate):
    N = len(arr1)
    
    control_0 = []
    control_1 = []
    target = None
    for i in range(N):
        
        # if ith qubits have same state -> control qubits
        if arr1[i] == arr2[i]:
            if arr1[i] == 0:
                control_0.append(i)
            else:
                control_1.append(i)
        else:
            target = i
        
    return FCGate(control_0, control_1, target, sub_gate)

def sub_gate_from_two_level_matrix(matrix, atol=0):

    """
    Return a Phase XZ decomposition of a 2x2 unitary matrix
    matrix - 2x2 unitary matrix
    atol - limit on the amount of error introduced by the construction.
    """

    sub_gate = cirq.optimizers.single_qubit_matrix_to_phxz(matrix, atol)

    if sub_gate != None:
        return sub_gate

    else:
        #TODO: Decide how to handle identity case
        print("Identity")
        
def create_gates_from_gray(matrix, indices, num_qubits) -> list:
    
    gray_arrs = create_gray_arrs(indices, num_qubits)
    N = len(gray_arrs)
    
    # Pre-ops: Apply FC_X gates   
    pre_gates = [state1_to_state2_gate(gray_arrs[i], gray_arrs[i+1], cirq.X) 
                 for i in range(N-2)]

    # Main op: Apply FC_U gate
    sub_gate = sub_gate_from_two_level_matrix(matrix)
    main_gate = [state1_to_state2_gate(gray_arrs[N-2], gray_arrs[N-1], sub_gate)]

    # Post-ops: Reverse pre-ops
    post_gates = list(gate for gate in pre_gates[::-1])

    return pre_gates + main_gate + post_gates

hide_toggle()

# Step 3: Create list of operations

In [126]:
def convert_unitary_to_ops(unitary, qubits):
    
    num_qubits = len(qubits)
    
    # Step 1: decompose unitary to two-level matrices
    matrices, indices_list = two_level_decomp(unitary)

    # Step 2: Create gates
    all_gates = []
    for matrix, indices in zip(matrices, indices_list):
        all_gates += create_gates_from_gray(matrix, indices, num_qubits)
        
    # Step 3: Create ops
    ops = [gate.build_op(qubits) for gate in all_gates]
    
    return ops

In [128]:
# Example
ops = convert_unitary_to_ops(A, qubits)
circuit = Circuit(ops)
print(circuit.to_text_diagram(transpose=True))

(0, 0)                          (0, 1)                           (0, 2)                           (0, 3)
│                               │                                │                                │
@───────────────────────────────@────────────────────────────────@────────────────────────────────PhXZ(a=-0.14,x=0.444,z=0.444)
│                               │                                │                                │
@───────────────────────────────@────────────────────────────────(0)──────────────────────────────X
│                               │                                │                                │
@───────────────────────────────@────────────────────────────────PhXZ(a=-0.881,x=0.7,z=0.762)─────(0)
│                               │                                │                                │
@───────────────────────────────@────────────────────────────────(0)──────────────────────────────X
│                               │                                

In [130]:
res = sim.simulate(circuit)
res.dirac_notation()

'(0.09+0.28j)|0000⟩ + (-0.19+0.13j)|0001⟩ + 0.17j|0010⟩ + (-0.2+0.07j)|0011⟩ + (-0.22-0.12j)|0100⟩ + (0.08-0.07j)|0101⟩ + (-0.33+0.02j)|0110⟩ + 0.2|0111⟩ + (0.17-0.15j)|1000⟩ + (-0.17-0.13j)|1001⟩ + (0.24-0.13j)|1010⟩ + (-0.09+0.09j)|1011⟩ + (-0.09+0.19j)|1100⟩ + (0.28+0.43j)|1101⟩ + (-0.12+0.17j)|1110⟩ + (-0.01-0.16j)|1111⟩'

# Example

### Before

In [7]:
qs = [cirq.GridQubit(0,1), cirq.GridQubit(1,1), cirq.GridQubit(2,1), cirq.GridQubit(2,2)] 

circuit = cirq.Circuit([cirq.X.on_each(qs[2]), cirq.I.on_each(qs[1], qs[0], qs[3])])
res = sim.simulate(circuit)
res.dirac_notation()

'|0010⟩'

### After

In [8]:
# Instantiate an FCgate
control_0 = [0, 3]
control_1 = [2]
target = 1 
sub_gate = cirq.X
fc_gate = FCGate(control_0, control_1, target, sub_gate)

circuit = cirq.Circuit([ 
    cirq.X.on(qs[2]), 
    fc_gate.build_gate(qs)
])
res = sim.simulate(circuit)
res.dirac_notation()

'|0110⟩'